In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cấu hình hiển thị: Giới hạn số cột hiển thị để bảng không bị quá dài
pd.set_option('display.max_columns', None)

Giải thích:

Chúng ta import pandas (thường viết tắt là pd) để làm việc với dữ liệu dạng bảng (DataFrame).

StandardScaler là công cụ toán học giúp đưa các cột dữ liệu về cùng một phân phối chuẩn (Z-score).

In [11]:
# Đường dẫn đến file dữ liệu
file_path = '../data/data.csv'

try:
    df = pd.read_csv(file_path)
    print(" Đã đọc file thành công.")
    print(f"Kích thước dữ liệu gốc: {df.shape}")
except FileNotFoundError:
    print(f" Không tìm thấy file tại: {file_path}")
    print("Vui lòng kiểm tra lại cấu trúc thư mục.")

 Đã đọc file thành công.
Kích thước dữ liệu gốc: (569, 33)


Giải thích:

Hàm pd.read_csv() đọc file data.csv từ thư mục data nằm ở cấp cha (..).

df.shape giúp ta biết ngay lập tức bảng dữ liệu có bao nhiêu dòng và bao nhiêu cột.

In [12]:
# 1. Loại bỏ cột 'Unnamed: 32' (thường sinh ra do lỗi dấu phẩy thừa cuối dòng trong file CSV)
if 'Unnamed: 32' in df.columns:
    df = df.drop(columns=['Unnamed: 32'])

# 2. Loại bỏ cột 'id' (Mã định danh bệnh nhân không giúp ích cho việc dự đoán bệnh)
if 'id' in df.columns:
    df = df.drop(columns=['id'])

# Kiểm tra lại 5 dòng đầu sau khi xóa
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Giải thích:

Cột Unnamed: 32 chứa toàn giá trị NaN (rỗng), cần xóa để không gây nhiễu.

Cột id là ngẫu nhiên và duy nhất cho mỗi bệnh nhân, giữ lại sẽ làm mô hình bị sai lệch (học vẹt), nên cần loại bỏ.

In [13]:
# Kiểm tra phân phối trước khi map
print("Nhãn trước khi mã hóa:", df['diagnosis'].unique())

# Chuyển đổi: M (Malignant - Ác tính) -> 1, B (Benign - Lành tính) -> 0
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

# Kiểm tra kết quả
print("Nhãn sau khi mã hóa:", df['diagnosis'].unique())

Nhãn trước khi mã hóa: ['M' 'B']
Nhãn sau khi mã hóa: [1 0]


Giải thích:

M (Malignant): Ác tính, là trường hợp chúng ta cần quan tâm phát hiện, nên gán là 1 (Positive Class).

B (Benign): Lành tính, gán là 0 (Negative Class).

In [14]:
# Tách riêng biến mục tiêu (y) và các đặc trưng đầu vào (X)
X = df.drop(columns=['diagnosis'])
y = df['diagnosis']

# Khởi tạo StandardScaler
scaler = StandardScaler()

# Thực hiện tính toán (fit) và biến đổi (transform)
# Kết quả trả về là một mảng Numpy (mất tiêu đề cột)
X_scaled_array = scaler.fit_transform(X)

# Chuyển mảng Numpy ngược lại thành DataFrame để giữ tên cột
X_scaled_df = pd.DataFrame(X_scaled_array, columns=X.columns)

# Ghép lại cột diagnosis (y) vào dữ liệu đã chuẩn hóa để tạo thành bộ dữ liệu hoàn chỉnh
df_processed = pd.concat([y, X_scaled_df], axis=1)

print(" Đã chuẩn hóa xong dữ liệu.")

 Đã chuẩn hóa xong dữ liệu.


fit_transform: Tính trung bình và độ lệch chuẩn của từng cột, sau đó áp dụng công thức z = (x - muy)/sigma.  
muy = (sum xi)/N.  
sigma = Căn bậc 2 (sum i=1 -> N (xi - muy)^2 / N).
Sau bước này, giá trị của radius_mean (vốn khoảng 10-20) và area_mean (vốn khoảng 500-2000) sẽ đều nằm trong khoảng nhỏ (ví dụ từ -3 đến +3), giúp thuật toán công bằng với cả 2 biến.

In [15]:
# 1. Xem 5 dòng đầu
print("Dữ liệu sau xử lý:")
display(df_processed.head())

# 2. Kiểm tra thống kê mô tả (Quan trọng: mean phải xấp xỉ 0, std phải xấp xỉ 1)
print("\nThống kê mô tả (Mean ~ 0, Std ~ 1):")
display(df_processed.describe().round(2))

# 3. Lưu ra file
output_path = '../data/processed_data.csv'
df_processed.to_csv(output_path, index=False)
print(f"\n Đã lưu dữ liệu đã xử lý vào file: {output_path}")

Dữ liệu sau xử lý:


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,2.255747,2.489734,-0.565265,2.833031,2.487578,-0.214002,1.316862,0.724026,0.660820,1.148757,0.907083,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015
1,1,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,-0.868652,0.499255,-0.876244,0.263327,0.742402,-0.605351,-0.692926,-0.440780,0.260162,-0.805450,-0.099444,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190
2,1,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,-0.398008,1.228676,-0.780083,0.850928,1.181336,-0.297005,0.814974,0.213076,1.424827,0.237036,0.293559,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391
3,1,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,4.910919,0.326373,-0.110409,0.286593,-0.288378,0.689702,2.744280,0.819518,1.115007,4.732680,2.047511,-0.281464,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010
4,1,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,-0.562450,1.270543,-0.790244,1.273189,1.190357,1.483067,-0.048520,0.828471,1.144205,-0.361092,0.499328,1.298575,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100



Thống kê mô tả (Mean ~ 0, Std ~ 1):


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00
mean,0.37,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00
std,0.48,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,0.00,-2.03,-2.23,-1.98,-1.45,-3.11,-1.61,-1.11,-1.26,-2.74,-1.82,-1.06,-1.55,-1.04,-0.74,-1.78,-1.30,-1.06,-1.91,-1.53,-1.10,-1.73,-2.22,-1.69,-1.22,-2.68,-1.44,-1.31,-1.75,-2.16,-1.60
25%,0.00,-0.69,-0.73,-0.69,-0.67,-0.71,-0.75,-0.74,-0.74,-0.70,-0.72,-0.62,-0.69,-0.62,-0.49,-0.62,-0.69,-0.56,-0.67,-0.65,-0.59,-0.67,-0.75,-0.69,-0.64,-0.69,-0.68,-0.76,-0.76,-0.64,-0.69
50%,0.00,-0.22,-0.10,-0.24,-0.30,-0.03,-0.22,-0.34,-0.40,-0.07,-0.18,-0.29,-0.20,-0.29,-0.35,-0.22,-0.28,-0.20,-0.14,-0.22,-0.23,-0.27,-0.04,-0.29,-0.34,-0.05,-0.27,-0.22,-0.22,-0.13,-0.22
75%,1.00,0.47,0.58,0.50,0.36,0.64,0.49,0.53,0.65,0.53,0.47,0.27,0.47,0.24,0.11,0.37,0.39,0.34,0.47,0.36,0.29,0.52,0.66,0.54,0.36,0.60,0.54,0.53,0.71,0.45,0.45
max,1.00,3.97,4.65,3.98,5.25,4.77,4.57,4.24,3.93,4.48,4.91,8.91,6.66,9.46,11.04,8.03,6.14,12.07,6.65,7.07,9.85,4.09,3.89,4.29,5.93,3.96,5.11,4.70,2.69,6.05,6.85



 Đã lưu dữ liệu đã xử lý vào file: ../data/processed_data.csv


Giải thích:

display(): Trong Jupyter, lệnh này hiển thị bảng đẹp hơn print().

Nhìn vào bảng describe: Sẽ thấy hàng mean toàn là 0.00 và hàng std toàn là 1.00. Điều này xác nhận quá trình chuẩn hóa thành công.

index=False: Khi lưu file CSV, ta không lưu cột số thứ tự dòng (0, 1, 2...) để file gọn nhẹ hơn.

In [16]:
from sklearn.model_selection import train_test_split

# 1. Xác định Features (X) và Target (y)
X = df_processed.drop(columns=['diagnosis'])
y = df_processed['diagnosis']

# 2. Bước 1: Tách tập Test (15% của tổng dữ liệu)
# Phần còn lại là Train + Val chiếm 85%
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, 
    test_size=0.15,          # 15% cho Test
    random_state=42, 
    stratify=y
)

# 3. Bước 2: Tách tập Validation (15% của tổng dữ liệu) từ tập còn lại
# Tập còn lại (X_train_val) đang chiếm 85% tổng dữ liệu.
# Muốn Validation chiếm 15% tổng, thì tỷ lệ cắt trong bước này phải là: 15 / 85
val_size = 0.15 / 0.85  # ~ 0.1765

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, 
    test_size=val_size,      # Tỷ lệ tương đối để lấy được 15% gốc
    random_state=42, 
    stratify=y_train_val
)

# 4. Kiểm tra kết quả
print(f"Tổng số mẫu: {len(df_processed)}")
print(f" Train set: {len(X_train)} mẫu ({len(X_train)/len(df_processed)*100:.2f}%) ~ 70%")
print(f" Val set:   {len(X_val)} mẫu ({len(X_val)/len(df_processed)*100:.2f}%) ~ 15%")
print(f" Test set:  {len(X_test)} mẫu ({len(X_test)/len(df_processed)*100:.2f}%) ~ 15%")

Tổng số mẫu: 569
 Train set: 397 mẫu (69.77%) ~ 70%
 Val set:   86 mẫu (15.11%) ~ 15%
 Test set:  86 mẫu (15.11%) ~ 15%


Giải thích chi tiết:

Train (Huấn luyện): Giống như sách giáo khoa để mô hình học kiến thức.

Validation (Kiểm thử): Giống như bài thi thử. Bạn dùng nó để tinh chỉnh tham số mô hình (hyperparameters) và chọn ra mô hình tốt nhất.

Test (Đánh giá): Giống như bài thi thật. Dữ liệu này được giấu kín hoàn toàn cho đến phút chót để đánh giá khách quan hiệu suất thực tế của mô hình.

stratify=y 

Đây là tham số cực kỳ quan trọng đối với dữ liệu y tế (thường bị mất cân bằng).

Nó đảm bảo rằng nếu dữ liệu gốc có 37% ác tính, thì cả 3 tập Train, Val, Test đều giữ đúng tỷ lệ 37% đó. Nếu không có nó, có thể xảy ra trường hợp tập Test toàn ca lành tính, dẫn đến đánh giá sai lầm.

random_state=42: Giúp cố định cách chia ngẫu nhiên. Nếu chạy lại code này 100 lần, kết quả chia vẫn giống hệt nhau, giúp kết quả thí nghiệm có thể tái lập được (reproducible).

In [17]:
import os

# Đường dẫn thư mục lưu trữ
save_dir = '../Vu_data/split_data'

# Tạo thư mục nếu chưa có
os.makedirs(save_dir, exist_ok=True)

# Lưu X (Features) và y (Target) cho từng tập
# Train
X_train.to_csv(f'{save_dir}/X_train.csv', index=False)
y_train.to_csv(f'{save_dir}/y_train.csv', index=False)

# Validation
X_val.to_csv(f'{save_dir}/X_val.csv', index=False)
y_val.to_csv(f'{save_dir}/y_val.csv', index=False)

# Test
X_test.to_csv(f'{save_dir}/X_test.csv', index=False)
y_test.to_csv(f'{save_dir}/y_test.csv', index=False)

print(f" Đã lưu 6 file dữ liệu vào thư mục: {save_dir}")

 Đã lưu 6 file dữ liệu vào thư mục: ../Vu_data/split_data


Giải thích:

Chúng ta tạo một thư mục con split_data để quản lý file gọn gàng.

Tách riêng X (đầu vào) và y (đầu ra) để làm việc tốt với các thư viện Deep Learning (như PyTorch, TensorFlow) hoặc Scikit-learn sau này.